In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool


In [2]:
llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-05T02:47:11.8482233Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 559706400, 'load_duration': 36454800, 'prompt_eval_count': 26, 'prompt_eval_duration': 157674000, 'eval_count': 25, 'eval_duration': 363467000}, id='run-79edd8e4-6028-47e8-b1c2-f332b130ede2-0', usage_metadata={'input_tokens': 26, 'output_tokens': 25, 'total_tokens': 51})

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri('sqlite:///data/employees.db')

In [4]:
db.dialect

'sqlite'

In [5]:
db.get_usable_table_names()

['departments', 'dept_emp', 'dept_manager', 'employees', 'salaries', 'titles']

In [6]:
db.run('SELECT COUNT(*) FROM "employees"')

'[(1,)]'

## Using llm chains

In [7]:
from langchain.chains import create_sql_query_chain

llm_chain = create_sql_query_chain(llm, db)

In [8]:
llm_chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [9]:
llm_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for (k, v) in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in t

In [10]:
question = 'how many employees are there?'
response = llm_chain.invoke({'question': question})
print(response)

Answer: There are 3 employees in the database.

SQLQuery:

SELECT COUNT(*) FROM employees


In [11]:
question = 'how many employees are there? You must return only sql SQLite query command.'
response = llm_chain.invoke({'question': question})
print(response)

Answer: There are 3 employees in the database.


In [12]:
db.run('SELECT COUNT(*) FROM employees')

'[(1,)]'

## create ask llm

In [13]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


base_url = "http://localhost:11434"
model = "llama3.2:3b"

llm = ChatOllama(base_url=base_url,
                 model=model,
                 temperature=0.1,
                 num_predict=256
                 )

system = SystemMessagePromptTemplate.from_template("""
You are a helpful AI assistant who answer user question based on provide context.
""")

prompt = """Answer user question based on the provided context ONLY! if do not know the answer, just sai "I don't know".
### Context:
{context}

### Question:
{question}

### Answer:
"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

qna_chain = template | llm | StrOutputParser()

def ask_llm(context, question):
    return qna_chain.invoke({'context':context, 'question':question}) 

In [14]:
from langchain_core.runnables import chain

@chain
def get_correct_sql_query(input):
    context = input['context']
    question = input['question']

    instruction = """
        Use above context to fetch the correct sql query for SQLite following question
        {}

        Do not enclose query in '''sql and do not write preamble and explanation.
        you must return only single SQL query  
    """.format(question)

    response = ask_llm(context=context, question=instruction)
    return response

In [15]:
correct = get_correct_sql_query.invoke(input={'context': response, 'question': question})

In [16]:
correct

'SELECT COUNT(*) FROM employees;'

In [17]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [18]:
execute_query = QuerySQLDataBaseTool(db=db)
sql_query = create_sql_query_chain(llm, db)

final_chain = (
    {"context": sql_query, 'question': RunnablePassthrough()}
    | get_correct_sql_query
    | execute_query
)

In [19]:
question = 'how many employees are there? You Must return only mysql Queries.'

response = final_chain.invoke({'question': question})
print(response)

[(1,)]


# Agents

In [20]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database. Given an input question, 
create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database. Only use the below tools.
Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it.
If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step. Then you should query the schema of the most relevant tables."""


In [21]:
from langchain_core.messages import SystemMessage

system_message = SystemMessage(content=SQL_PREFIX) 

In [22]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000215374A3EB0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000215374A3EB0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000215374A3EB0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [23]:
from langchain import hub
from langchain_core.messages import HumanMessage
from langchain.agents import create_react_agent

In [24]:
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

c:\Users\leand\miniconda3\envs\langchain\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [25]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT m

In [26]:
print(prompt_template.input_variables)

['dialect', 'top_k']


In [27]:
prompt = prompt_template.format(dialect="SQLite", top_k=5)

In [28]:
agent_executor = create_react_agent(llm, tools, prompt)

AttributeError: 'str' object has no attribute 'input_variables'

In [ ]:
question = 'how many employees are there?'

agent_executor.invoke({'messages': [HumanMessage(content=question)]})